In [1]:
import pandas as pd

data = pd.read_csv('weather_data_danang_15k.csv')

Work flow
1. Xử lý dữ liệu
 - Scale dữ liệu bằng RobustScaler
 - Chuyển đổi cột 'time' sang định dạng datetime
 - Xử lý các giá trị thiếu (nếu có)
 - Dùng biến trễ Lag features: dữ liệu thời tiết của các giờ trước đó để dự đoán giờ hiện tại, ngày hôm trước để dự đoán ngày hôm nay,...
 - Dùng Rolling Mean/Median/Max/Min/Std: tính toán các giá trị trung bình/trung vị/lớn nhất/nhỏ nhất/độ lệch chuẩn trong một khoảng thời gian cố định để làm mượt dữ liệu và giảm nhiễu. Giúp mô hình học được xu hướng dài hạn trong dữ liệu thời tiết.
2. Chia dữ liệu thành train, val, test (70%, 20%, 10%)
3. Sử dụng mô hình AdaBoost, Softmax Regresssion
RamdomForest train trên tập train, tinh chỉnh hyperparameter trên tập val
4. Trực quan hóa kết quả huấn luyện
5. Đánh giá mô hình bằng accuracy, precision, recall, F1-score, confusion matrix, Cohen's Kappa, Confusion Matrix Heatmap
6. So sánh kết quả giữa các mô hình

In [2]:
data.head()

,time,temperature_2m (°C),weather_code (wmo code),relative_humidity_2m (%),surface_pressure (hPa),wind_speed_10m (km/h),wind_direction_10m (°),cloud_cover (%)
0,2025-01-01T00:00,20.1,2,90,1016.1,3.8,295,65
1,2025-01-01T01:00,19.8,2,92,1015.4,4.4,291,72
2,2025-01-01T02:00,19.2,2,94,1014.5,5.0,285,53
3,2025-01-01T03:00,18.8,3,94,1014.1,3.7,284,91
4,2025-01-01T04:00,18.7,1,94,1013.9,2.8,288,42


In [ ]:
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8880 entries, 0 to 8879
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   time                      8880 non-null   object 
 1   temperature_2m (°C)       8880 non-null   float64
 2   weather_code (wmo code)   8880 non-null   int64  
 3   relative_humidity_2m (%)  8880 non-null   int64  
 4   surface_pressure (hPa)    8880 non-null   float64
 5   wind_speed_10m (km/h)     8880 non-null   float64
 6   wind_direction_10m (°)    8880 non-null   int64  
 7   cloud_cover (%)           8880 non-null   int64  
dtypes: float64(3), int64(4), object(1)
memory usage: 555.1+ KB


In [4]:

data.describe()

,temperature_2m (°C),weather_code (wmo code),relative_humidity_2m (%),surface_pressure (hPa),wind_speed_10m (km/h),wind_direction_10m (°),cloud_cover (%)
count,8880.000000,8880.000000,8880.000000,8880.000000,8880.000000,8880.000000,8880.000000
mean,25.856813,20.614077,81.532995,1009.261239,8.744921,172.609685,81.385586
std,3.984711,24.763184,12.641720,5.595204,5.708659,106.303039,30.683129
min,16.100000,0.000000,31.000000,993.900000,0.000000,0.000000,0.000000
25%,23.000000,3.000000,74.000000,1005.000000,4.400000,79.000000,74.000000
50%,25.600000,3.000000,85.000000,1009.000000,7.600000,169.000000,100.000000
75%,28.400000,51.000000,92.000000,1013.800000,12.000000,252.000000,100.000000
max,40.200000,65.000000,100.000000,1025.100000,37.600000,360.000000,100.000000


In [5]:
data.head()

,time,temperature_2m (°C),weather_code (wmo code),relative_humidity_2m (%),surface_pressure (hPa),wind_speed_10m (km/h),wind_direction_10m (°),cloud_cover (%)
0,2025-01-01T00:00,20.1,2,90,1016.1,3.8,295,65
1,2025-01-01T01:00,19.8,2,92,1015.4,4.4,291,72
2,2025-01-01T02:00,19.2,2,94,1014.5,5.0,285,53
3,2025-01-01T03:00,18.8,3,94,1014.1,3.7,284,91
4,2025-01-01T04:00,18.7,1,94,1013.9,2.8,288,42


In [ ]:
data1 = data.drop(columns=['weather_code (wmo code)']) 

##Robust Scaler

In [7]:
from sklearn.preprocessing import RobustScaler
numeric_features = ['temperature_2m (°C)','relative_humidity_2m (%)','surface_pressure (hPa)',
                    'wind_speed_10m (km/h)','wind_direction_10m (°)', 'cloud_cover (%)',]

robust_scaler = RobustScaler()
data_scaled = data1.copy()
data_scaled[numeric_features] = robust_scaler.fit_transform(data[numeric_features])

                    

In [8]:
data_scaled.describe()

,temperature_2m (°C),relative_humidity_2m (%),surface_pressure (hPa),wind_speed_10m (km/h),wind_direction_10m (°),cloud_cover (%)
count,8880.000000,8880.000000,8880.000000,8880.000000,8880.000000,8880.000000
mean,0.047558,-0.192611,0.029686,0.150648,0.020865,-0.715939
std,0.737909,0.702318,0.635819,0.751139,0.614468,1.180120
min,-1.759259,-3.000000,-1.715909,-1.000000,-0.976879,-3.846154
25%,-0.481481,-0.611111,-0.454545,-0.421053,-0.520231,-1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.518519,0.388889,0.545455,0.578947,0.479769,0.000000
max,2.703704,0.833333,1.829545,3.947368,1.104046,0.000000


In [9]:
data_scaled.head()

,time,temperature_2m (°C),relative_humidity_2m (%),surface_pressure (hPa),wind_speed_10m (km/h),wind_direction_10m (°),cloud_cover (%)
0,2025-01-01T00:00,-1.018519,0.277778,0.806818,-0.500000,0.728324,-1.346154
1,2025-01-01T01:00,-1.074074,0.388889,0.727273,-0.421053,0.705202,-1.076923
2,2025-01-01T02:00,-1.185185,0.500000,0.625000,-0.342105,0.670520,-1.807692
3,2025-01-01T03:00,-1.259259,0.500000,0.579545,-0.513158,0.664740,-0.346154
4,2025-01-01T04:00,-1.277778,0.500000,0.556818,-0.631579,0.687861,-2.230769


In [10]:
data_scaled['weather_code (wmo code)'] = data['weather_code (wmo code)']

In [11]:
data_scaled.head()

,time,temperature_2m (°C),relative_humidity_2m (%),surface_pressure (hPa),wind_speed_10m (km/h),wind_direction_10m (°),cloud_cover (%),weather_code (wmo code)
0,2025-01-01T00:00,-1.018519,0.277778,0.806818,-0.500000,0.728324,-1.346154,2
1,2025-01-01T01:00,-1.074074,0.388889,0.727273,-0.421053,0.705202,-1.076923,2
2,2025-01-01T02:00,-1.185185,0.500000,0.625000,-0.342105,0.670520,-1.807692,2
3,2025-01-01T03:00,-1.259259,0.500000,0.579545,-0.513158,0.664740,-0.346154,3
4,2025-01-01T04:00,-1.277778,0.500000,0.556818,-0.631579,0.687861,-2.230769,1


In [13]:
# Chuyển đổi sang datetime
data_scaled['time'] = pd.to_datetime(data_scaled['time'])

##AdaBoost Classifier

In [14]:
from sklearn.model_selection import train_test_split
X = data_scaled.drop(columns=['weather_code (wmo code)'])
y = data_scaled['weather_code (wmo code)']

X_temp, X_val, y_temp, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.125, random_state=42)

train: 80 --> train :70, test:10
val=20


In [ ]:

from sklearn.ensemble import AdaBoostClassifier
ada_reg = AdaBoostClassifier(n_estimators=100, random_state=42)
ada_reg.fit(X_train, y_train)
y_pred = ada_reg.predict(X_test)
